# Introduction to Torch
- Deep learning library developed by Meta (Formerly Facebook)
- Is a key component of PyTorch ecosystem

Similarities with Autograd:
- Automatic differentiation
- Easy of use
- Pythonic integration
- Dynamic computational graphs
- Gradient computation and optimization

Differences with AutoGrad:
- Based on **tensors** (multidimensional matrixes) instead of **values**
- Easy to execute on GPUs of different vendors
- Lots of components and tools integrated

## A simple example in torch

In [ ]:
import torch

x1 = torch.Tensor([2.0]).double(); x1.requires_grad = True
x2 = torch.Tensor([0.0]).double(); x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double(); w1.requires_grad = True
w2 = torch.Tensor([1.0]).double(); w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double(); b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())

In [ ]:
o.backward()

print('x1', x1.grad.item())
print('w1', w1.grad.item())
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('b', b.grad.item())

## A neural network in torch

In [ ]:
xs = torch.tensor([
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, -1.0],
    [1.0, 1.0, -1.0]
])
ys = torch.tensor([1.0, -1.0, -1.0, 1.0])

In [ ]:
# Neuron, three inputs
W = torch.randn((3, 1))
W

In [ ]:
# activation of the neuron on the four inputs
xs @ W

In [ ]:
# Lets now add the bias
b = torch.randn((1,))
b

In [ ]:
# activation of the first neuron for the four inputs
xs @ W + b

In [ ]:
# lets add more neurons
W = torch.randn((3, 4))
W

In [ ]:
# what is now the product?
xs @ W

In [ ]:
# xs -> objects x features
# W -> weights x neurons
# xs @ W -> objects x neurons, each cell contains sum(features*weights)
xs.shape, W.shape, (xs@W).shape

In [ ]:
# the bias is added per neuron, after xs * W
b = torch.randn((1,4))
act = xs @ W + b
display(xs @ W)
display(b)
display(act)

In [ ]:
# now, it is time to evaluate the activation function on the activation
(xs @ W + b).tanh()

As a summary, a fully connected layer is formed by:
- A weight matrix that has one row per input value, and a column per output value
- A bias vector, that has a single row and a column per neuron
- An activation function

Note: All the elements here are differentiable.

In [ ]:
# back to the original problem
display(xs)
display(ys)

In [ ]:
# we have three features and one output, so we need one output neuron
W = torch.randn((3, 1))
b = torch.randn((1, 1))

out = (xs @ W + b).tanh()
out

In [ ]:
# lets calculate the loss
loss = torch.sum((out - ys)**2)
loss

In [ ]:
# Uppps, it should not be that large!
out-ys

In [ ]:
out.shape, ys.shape, (out-ys).shape

In [ ]:
# one solution is remove the last dimension from out
out.shape, out.squeeze().shape

In [ ]:
out.squeeze() - ys

In [ ]:
loss = torch.sum((out.squeeze() - ys)**2)
loss

In [ ]:
# since everything is differentiable, we can backpropagate the loss
loss.backward()

In [ ]:
# In torch, we need to warn the framework about the tensors we need to calculate the grad

# network definition, seed controled
g = torch.Generator().manual_seed(31416)
W = torch.randn((3, 1), generator=g, requires_grad=True)
b = torch.randn((1, 1), generator=g, requires_grad=True)

In [ ]:
learning_rate = 0.1
for _ in range(100):
    # forward pass
    out = (xs @ W + b).tanh()
    loss = torch.mean((out.squeeze() - ys)**2)
#     print(loss.item())

    # backward pass, need to remove the gradients
    W.grad = None
    b.grad = None
    loss.backward()

    # update parameters
    W.data += -learning_rate * W.grad
    b.data += -learning_rate * b.grad
print("Final loss:", loss.item())

In [ ]:
out.data, ys

In [ ]:
g = torch.Generator().manual_seed(31416)
W = torch.randn((3, 1), generator=g, requires_grad=True)
b = torch.randn((1, 1), generator=g, requires_grad=True)

In [ ]:
# Let simplify the code
learning_rate = 0.1
parameters = [W, b]
for _ in range(100):
    # forward pass
    out = (xs @ W + b).tanh()
    loss = torch.mean((out.squeeze() - ys)**2)
    print(loss.item())

    # backward pass, need to remove the gradients
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    for p in parameters:
        p.data += -learning_rate * p.grad
        
print("Final loss:", loss.item())


In [ ]:
out.data, ys

## Solving moon problem, using the GPU

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
np.random.seed(1337)
random.seed(1337)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=100, noise=0.1)
# visualize in 2D
plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')
plt.show()

In [ ]:
# Lets transform them to tensors, and keep only the first to simplify the explanations
xs = torch.tensor(X, dtype=torch.float32, device=device)[:5]
ys = torch.tensor(y, device=device)[:5]
xs.shape, ys.shape
ys.device

In [ ]:
xs, ys

In [ ]:
# Now, lets create a neural network having two hidden layers, with 4 and 5 neurons
# Since it is a classification problem, we will use one output layer per class.

In [ ]:
g = torch.Generator(device=device).manual_seed(31416+2)
W1 = torch.randn((2, 4), generator=g, requires_grad=True, device=device)
b1 = torch.randn((1, 4), generator=g, requires_grad=True, device=device)

W2 = torch.randn((4, 5), generator=g, requires_grad=True, device=device)
b2 = torch.randn((1, 5), generator=g, requires_grad=True, device=device)

W3 = torch.randn((5, 2), generator=g, requires_grad=True, device=device)
b3 = torch.randn((1, 2), generator=g, requires_grad=True, device=device)

parameters = [W1, b1, W2, b2, W3, b3]

In [ ]:
# forward pass
l1_out = (xs @ W1 + b1).tanh()
l2_out = (l1_out @ W2 + b2).tanh()
l3_out = (l2_out @ W3 + b3)
l3_out


In [ ]:
l3_out.device

We want the output of the network to be class probabilities, so we need to transform the outputs. To be considered as a probability, values must between 0 and 1, and they must add to 1.

First, lets apply the exponencial function, to turn all values positive.

In [ ]:
l3_out.exp()

Now, re-scale the values so they add 1

In [ ]:
value = l3_out.exp()
(value / value.sum(dim=1))

In [ ]:
# Lets explore the cause of the error
value.shape, value.sum(dim=1).shape

The problem happens because of the way broadcast is performed:
- Align the dimension from right to left
- Fill missing dimensions with 1
- Dimensions are compatible if they have the same size, or all but one contains value 1
- Broadcast the values in the values in the vector with 1 as necessary

Examples:
- 5x1, 1x5 -> 5x5
- 6x3, 3 -> 6x3 , 1x3 -> 6x3

In the line with the error, torch try to broadcast 5x2 and 5 -> 5x2, 1x5, and they are not compatible.

There are some solutions. Here we use the unsqueeze() method that adds a dimension to the tensor, in the **dim** position.

In [ ]:
value = l3_out.exp()
probs = (value / value.sum(dim=1).unsqueeze(dim=1))
probs[:5]

In [ ]:
# Safety check
probs.sum(dim=1)

Now we have probabilities as the output of the network. How to create a loss function, using this probabilities.

One commonly used idea is to use the maximal likelihood estimation. It estimates the quality of the result by multiplying the probabilities obtained for the correct class.

In [ ]:
display(probs)
ys

In [ ]:
# In our case
probs[0, 0], probs[1, 1], probs[2, 0], probs[3, 1], probs[4,1]

In [ ]:
# How to obtain this probabilities using tensor manipulation: using a type of indexing
probs[torch.arange(len(ys)), ys]

In [ ]:
# lets multiply them
likelihood = probs[torch.arange(len(ys)), ys].prod()
likelihood

This is a quite small number, and we only have 5 results ... imagine when we have 100s

Solution, use the logarithms, and average the results

In [ ]:
log_likelihood = probs[torch.arange(len(ys)), ys].log().mean()
log_likelihood

Since likelihood is higher (so log_likelihood) in better results, and we need a loss function, we will invert the sign.

**Note:** All performed operations are derivables!

In [ ]:
nll = - log_likelihood
nll

Lets put it all together

In [ ]:
xs = torch.tensor(X, dtype=torch.float32, device=device)
ys = torch.tensor(y, device=device)

In [ ]:
g = torch.Generator(device=device).manual_seed(31416+2)
W1 = torch.randn((2, 4), generator=g, requires_grad=True, device=device)
b1 = torch.randn((1, 4), generator=g, requires_grad=True, device=device)

W2 = torch.randn((4, 5), generator=g, requires_grad=True, device=device)
b2 = torch.randn((1, 5), generator=g, requires_grad=True, device=device)

W3 = torch.randn((5, 2), generator=g, requires_grad=True, device=device)
b3 = torch.randn((1, 2), generator=g, requires_grad=True, device=device)

parameters = [W1, b1, W2, b2, W3, b3]

In [ ]:
learning_rate = 0.1
epochs = 2000

for epoch in range(epochs):
    # forward pass
    l1_out = (xs @ W1 + b1).tanh()
    l2_out = (l1_out @ W2 + b2).tanh()
    l3_out = (l2_out @ W3 + b3)
    value = l3_out.exp()
    probs = (value / value.sum(dim=1).unsqueeze(dim=1))
    log_likelihood = probs[torch.arange(len(ys)), ys].log().mean()
    loss = -log_likelihood
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, loss={loss.item()}")
    
    # backward pass, need to remove the gradients
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    for p in parameters:
        p.data += -learning_rate * p.grad
print(loss.item())

In [ ]:
# Lets calculate the classifier accuracy
probs[torch.arange(len(ys)), ys] > 0.5

The classifier commits no mistakes, so it's accuracy is 1.0. 

Lets plot the decision boundary

In [ ]:
h = 0.25
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
Xmesh[:10]

In [ ]:
xs_full = torch.tensor(Xmesh, device=device).float()
l1_out = (xs_full @ W1 + b1).tanh()
l2_out = (l1_out @ W2 + b2).tanh()
l3_out = (l2_out @ W3 + b3).tanh()
value = l3_out.exp()
probs = (value / value.sum(dim=1).unsqueeze(dim=1))
probs[:5]

In [ ]:
scores = torch.max(probs, dim=1).indices
scores

In [ ]:
Z = np.array([s.data > 0 for s in scores.cpu()])
Z = Z.reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

### Use modules in torch.nn

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

xs = torch.tensor(X, dtype=torch.float32, device=device)
ys = torch.tensor(y, device=device)

# Using torch fully connected layers
l1 = nn.Linear(in_features=2, out_features=4, device=device)
l2 = nn.Linear(in_features=4, out_features=5, device=device)
l3 = nn.Linear(in_features=5, out_features=2, device=device)
parameters = list(l1.parameters()) + list(l2.parameters()) + list(l3.parameters())

x = l1(xs).tanh()
x = l2(x).tanh()
x = l3(x)

# soft-max layer + entropy loss = cross_entropy
loss = F.cross_entropy(x, ys)
loss

In [ ]:
learning_rate = 0.1
epochs = 20000

for epoch in range(epochs):
    
    # forward pass
    x = l1(xs).tanh()
    x = l2(x).tanh()
    x = l3(x)
    loss = F.cross_entropy(x, ys)
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, loss={loss.item()}")
    
    # backward pass, need to remove the gradients
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    for p in parameters:
        p.data += -learning_rate * p.grad
        
print(loss.item())

Lets put in into a more compacted version, using Sequential, and an optimizer

In [ ]:
xs = torch.tensor(X, dtype=torch.float32, device=device)
ys = torch.tensor(y, device=device)

# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(2, 4, device=device), nn.Tanh(),
    nn.Linear(4, 5, device=device), nn.Tanh(),
    nn.Linear(5, 2, device=device)
)

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
epochs = 10000

for epoch in range(epochs):
    
    # Forward pass
    x = model(xs)
    loss = F.cross_entropy(x, ys)
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, loss={loss.item()}")
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Update parameters
    optimizer.step()
        
print(loss.item())


Here we used an optimizer, that performs the update operation faster and smartly, together with the zero_grad() operation.

There is other way to create the network, subclassing the Module class.

In [ ]:

# Define the custom model class
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(2, 4, device=device)
        self.layer2 = nn.Linear(4, 5, device=device)
        self.layer3 = nn.Linear(5, 2, device=device)

    def forward(self, xs):
        x = self.layer1(xs).tanh()
        x = self.layer2(x).tanh()
        x = self.layer3(x)
        return x

# Instantiate the model
model = MyModel()

xs = torch.tensor(X, dtype=torch.float32, device=device)
ys = torch.tensor(y, device=device)

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

epochs = 2000

for epoch in range(epochs):
    # Forward pass
    outputs = model(xs)
    loss = F.cross_entropy(outputs, ys)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, loss={loss.item()}")
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Update parameters
    optimizer.step()
        
print(loss.item())


# Some unfrequent uses of torch autograd

## KNN

In [ ]:
import torch

class KMeansTorch:
    def __init__(self, n_clusters, max_iters=100, tol=1e-4, lr=0.01):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.tol = tol
        self.lr = lr

    def fit(self, X):
        # Randomly initialize the centroids as k random samples from X
        indices = torch.randperm(X.size(0))[:self.n_clusters]
        self.centroids = X[indices]
        self.centroids.requires_grad = True  # Enable gradients for centroids

        optimizer = torch.optim.Adam([self.centroids], lr=self.lr)

        for i in range(self.max_iters):
            # Assign each point to the nearest centroid
            distances = torch.cdist(X, self.centroids, p=2)
            cluster_assignments = torch.argmin(distances, dim=1)

            # Compute the loss (sum of squared distances)
            loss = 0
            for k in range(self.n_clusters):
                loss += ((X[cluster_assignments == k] - self.centroids[k]) ** 2).sum()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Check for convergence (if centroids do not change significantly)
            with torch.no_grad():
                if i > 0 and prev_loss - loss.item() < self.tol:
                    break
                prev_loss = loss.item()

        self.labels_ = cluster_assignments.detach()
        return self

    def predict(self, X):
        distances = torch.cdist(X, self.centroids, p=2)
        return torch.argmin(distances, dim=1)

In [ ]:
# Function to generate 2D Gaussian data
def generate_2d_gaussian(mean, cov, num_samples):
    return np.random.multivariate_normal(mean, cov, num_samples)

# Means and covariances for the Gaussians
means = [[2, 2], [8, 3], [5, 7]]
covs = [[[1, 0.5], [0.5, 1]], [[1, -0.7], [-0.7, 1]], [[1, 0.3], [0.3, 1]]]

# Generate samples
num_samples = 100
data = np.vstack([generate_2d_gaussian(mean, cov, num_samples) for mean, cov in zip(means, covs)])

# Convert to PyTorch tensor
X = torch.tensor(data, dtype=torch.float32, device=device)


In [ ]:
# Plot the generated data points
plt.scatter(data[:, 0], data[:, 1], s=10)
plt.title("Generated 2D Gaussian Data")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()


In [ ]:
kmeans = KMeansTorch(n_clusters=3, max_iters=1000, lr=0.01)
kmeans.fit(X)

In [ ]:
# Plot the clustered data points
plt.scatter(data[:, 0], data[:, 1], c=kmeans.labels_.cpu(), s=10, cmap='viridis')
plt.scatter(kmeans.centroids[:, 0].detach().cpu().numpy(), 
            kmeans.centroids[:, 1].detach().cpu().numpy(), 
            s=100, c='red', label='Centroids')
plt.title("KMeans Clustering with PyTorch")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend()
plt.show()


## PCA

In [ ]:
# Generate synthetic data
np.random.seed(42)
data = np.random.randn(100, 5)
X = torch.tensor(data, dtype=torch.float32)

# Initialize the principal component vector (random)
w = torch.randn(X.size(1), requires_grad=True)

# Learning rate and iterations
lr = 0.01
iterations = 1000

# Gradient descent to find the principal component
for i in range(iterations):
    # Project the data onto the principal component vector
    projected = X @ w

    # Compute the loss (negative variance)
    loss = -torch.var(projected)

    # Backward pass and optimization
    loss.backward()

    with torch.no_grad():
        w += lr * w.grad
        w.grad.zero_()
    
    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {loss.item()}")

# Normalize the principal component vector
principal_component = w / torch.norm(w)
print("Principal Component:", principal_component.detach().numpy())


## Linear regression

In [ ]:
import torch

# Generate synthetic data
np.random.seed(42)
X_np = np.random.rand(100, 1)
y_np = 3 * X_np.squeeze() + 2 + 0.1 * np.random.randn(100)

X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# Initialize weights and bias
w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

# Learning rate and iterations
lr = 0.1
iterations = 1000

# Gradient descent for linear regression
for i in range(iterations):
    # Compute predictions
    y_pred = X @ w + b

    # Compute the loss (mean squared error)
    loss = F.mse_loss(y_pred, y)

    # Backward pass and optimization
    loss.backward()

    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad
        w.grad.zero_()
        b.grad.zero_()
    
    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {loss.item()}")

print("Learned weights:", w.item())
print("Learned bias:", b.item())


##